# $ Tidy Data $

#### Imports

In [14]:
import pandas as pd
import env
from pydataset import data
import acquire

# train test split from sklearn
from sklearn.model_selection import train_test_split
# imputer from sklearn
# help with missing value by replacing blank with: median, mode, average, calculate using other column
from sklearn.impute import SimpleImputer

db_url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/tidy_data'

### 1. Attendance Data

Read the data from the attendance table and calculate an attendance percentage for each student. One half day is worth 50% of a full day, and 10 tardies is equal to one absence.

You should end up with something like this:


name

Billy    0.5250 

Jane     0.6875

John     0.9125

Sally    0.7625

Name: grade, dtype: float64
Coffee Levels

In [38]:
db_url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/tidy_data'

   

sql_query = """
                SELECT * 
                From attendance;
                """

attendance = pd.read_sql(sql_query, db_url)

In [39]:
# check to see what attendance looks like
attendance

,Unnamed: 0,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [40]:
# rename columns to name
attendance.rename(columns={'Unnamed: 0':'name'}, inplace = True)

In [41]:
# create a dataframe where we melt data and attendance
df = attendance.melt(id_vars='name',var_name= 'date', value_name='attendance')
df

,name,date,attendance
0,Sally,2018-01-01,P
1,Jane,2018-01-01,A
2,Billy,2018-01-01,A
3,John,2018-01-01,P
4,Sally,2018-01-02,T
5,Jane,2018-01-02,P
6,Billy,2018-01-02,T
7,John,2018-01-02,T
8,Sally,2018-01-03,T
9,Jane,2018-01-03,T



alternative way using map

df.attendance.map({'P':1,'H':0.5,'A':0,'T':0.9})

In [42]:
# replace data values of attendance with floats relating to attendance
df['attendance'].replace({'A':0,'P':1,'T':.9,'H':0.5}, inplace = True)

In [43]:
df.sample(10)

,name,date,attendance
9,Jane,2018-01-03,0.9
15,John,2018-01-04,1.0
1,Jane,2018-01-01,0.0
0,Sally,2018-01-01,1.0
8,Sally,2018-01-03,0.9
31,John,2018-01-08,1.0
25,Jane,2018-01-07,0.0
30,Billy,2018-01-08,0.9
23,John,2018-01-06,0.9
5,Jane,2018-01-02,1.0


In [44]:
# group by name to obtain average attendance
attendance_avg = df.groupby('name').mean().astype('float')

In [45]:
# date is dropped as an algorithm of our groupby
attendance_avg.reset_index()

,name,attendance
0,Billy,0.5250
1,Jane,0.6875
2,John,0.9125
3,Sally,0.7625


### 2. Coffee Levels

### a) Read the coffee_levels table.

In [26]:
# read the coffee_levels table from codeup database
sql_query = """
                SELECT * 
                From coffee_levels;
                """

coffee_levels = pd.read_sql(sql_query, db_url )

In [28]:
coffee_levels.sample(10)


,hour,coffee_carafe,coffee_amount
6,14,x,0.507288
23,11,z,0.311495
9,17,x,0.391560
19,17,y,0.594126
26,14,z,0.864464
15,13,y,0.997464
29,17,z,0.436677
22,10,z,0.144928
5,13,x,0.310711
4,12,x,0.898291


### b) Transform the data so that each carafe is in it's own column.

In [29]:
# Transform the data so that each carafe is its own columns
coffee_levels.pivot_table(index=['hour'],
                         columns = 'coffee_carafe',
                         values = 'coffee_amount').reset_index().rename_axis(None, axis = 1)

,hour,x,y,z
0,8,0.816164,0.189297,0.999264
1,9,0.451018,0.521502,0.915990
2,10,0.843279,0.023163,0.144928
3,11,0.335533,0.235529,0.311495
4,12,0.898291,0.017009,0.771947
5,13,0.310711,0.997464,0.398520
6,14,0.507288,0.058361,0.864464
7,15,0.215043,0.144644,0.436364
8,16,0.183891,0.544676,0.280621
9,17,0.391560,0.594126,0.436677


### c) Is this the best shape for the data?

The original is the best because each variable is in each column and each row is a single observation

### 3. Cake Recipes

### a) Read the cake_recipes table. This data set contains cake tastiness scores for combinations of different recipes, oven rack positions, and oven temperatures.

In [30]:
# read the coffee_levels table from codeup database
sql_query = """
                SELECT * 
                From cake_recipes;
                """

cake_recipes = pd.read_sql(sql_query, db_url)

In [31]:
# see the data, recipe:position should be seperated, temp values should be its own column
cake_recipes

,recipe:position,225,250,275,300
0,a:bottom,61.738655,53.912627,74.414730,98.786784
1,a:top,51.709751,52.009735,68.576858,50.228470
2,b:bottom,57.095320,61.904369,61.196980,99.248541
3,b:top,82.455004,95.224151,98.594881,58.169349
4,c:bottom,96.470207,52.001358,92.893227,65.473084
5,c:top,71.306308,82.795477,92.098049,53.960273
6,d:bottom,52.799753,58.670419,51.747686,56.183110
7,d:top,96.873178,76.101363,59.571620,50.971626


### b) Tidy the data as necessary.

In [32]:
# split recipe and position to create individual observations, set expand = True so each variable gets its own columns
recipe_position_df = cake_recipes['recipe:position'].str.split(':', expand= True)
recipe_position_df.columns =['recipe', 'position']
recipe_position_df.head()

,recipe,position
0,a,bottom
1,a,top
2,b,bottom
3,b,top
4,c,bottom


In [33]:
# dropped concat original dataframe with split variable dataframe and drop column
cake2=pd.concat([recipe_position_df,cake_recipes], axis = 1).drop(columns = 'recipe:position')
cake2.head()

,recipe,position,225,250,275,300
0,a,bottom,61.738655,53.912627,74.414730,98.786784
1,a,top,51.709751,52.009735,68.576858,50.228470
2,b,bottom,57.095320,61.904369,61.196980,99.248541
3,b,top,82.455004,95.224151,98.594881,58.169349
4,c,bottom,96.470207,52.001358,92.893227,65.473084


In [34]:
# create a melt and keep vairables as columns ['recipe','position'] and convert the rest to 'temp'column, use values as rate values
cake_melt = cake2.melt(id_vars=['recipe','position'],
                 var_name='temp',
                 value_name='rate')

In [35]:
# tidy data
cake_melt

,recipe,position,temp,rate
0,a,bottom,225,61.738655
1,a,top,225,51.709751
2,b,bottom,225,57.095320
3,b,top,225,82.455004
4,c,bottom,225,96.470207
5,c,top,225,71.306308
6,d,bottom,225,52.799753
7,d,top,225,96.873178
8,a,bottom,250,53.912627
9,a,top,250,52.009735


### c) Which recipe, on average, is the best? 
Recipe b has the highest mean 76.73607

In [49]:
# group by recipe type and calculate the mean
cake_melt.groupby('recipe').mean().idxmax()

rate    b
dtype: object

In [50]:
cake_melt.groupby('recipe').mean().max()

rate    76.736074
dtype: float64

use idxmax() to show the max of what was grouped by
use max() to show the max rate

### d) Which oven temperature, on average, produces the best results? 
The oven temperature of 275 producees the best results according to the rating of 74.886754


In [51]:
cake_melt.groupby('temp').mean()

,rate
temp,
225,71.306022
250,66.577437
275,74.886754
300,66.627655


In [52]:
# to show the exact temp that is the max
cake_melt.groupby('temp').mean().idxmax()

rate    275
dtype: object

In [54]:
# show max by using sort and tail shows the row of the max temp 
cake_melt.groupby('temp').mean().sort_values('rate').tail(1)

,rate
temp,
275,74.886754


### e) Which combination of recipe, rack position, and temperature gives the best result?
Recipe B in bottom position at 300 degrees has the highest rating at 99.248541

In [55]:
# bool mask to return a row of  given parameters
cake_melt[cake_melt.rate == cake_melt.rate.max()]

,recipe,position,temp,rate
26,b,bottom,300,99.248541


In [57]:
cake_melt.groupby(['recipe','position','temp']).mean()

rate
recipe position temp           
a      bottom   225   61.738655
                250   53.912627
                275   74.414730
                300   98.786784
       top      225   51.709751
                250   52.009735
                275   68.576858
                300   50.228470
b      bottom   225   57.095320
                250   61.904369
                275   61.196980
                300   99.248541
       top      225   82.455004
                250   95.224151
                275   98.594881
                300   58.169349
c      bottom   225   96.470207
                250   52.001358
                275   92.893227
                300   65.473084
       top      225   71.306308
                250   82.795477
                275   92.098049
                300   53.960273
d      bottom   225   52.799753
                250   58.670419
                275   51.747686
                300   56.183110
       top      225   96.873178
                250   76.101363
                275   59.571620
                300   50.971626

In [58]:
# using idxmax()
cake_melt.groupby(['recipe','position','temp']).mean().idxmax()

rate    (b, bottom, 300)
dtype: object

In [60]:
# using sort and tail
cake_melt.groupby(['recipe','position','temp']).mean().sort_values('rate').tail(1)

,,,rate
recipe,position,temp,
b,bottom,300,99.248541


# 4 Bonus: explore the other tables in the tidy_data database and reshape them as necessary so that they are in a tidy format.

In [ ]:
# read the coffee_levels table from codeup database
sql_query = """
                SELECT * 
                From gapminder1;
                """

gapminder1 = pd.read_sql(sql_query, db_url)

In [ ]:
gapminder1.sample(10)

 Measure pop, fertility, life_expect look like variables with measurement as values,
 So we need to go up ...pivot table

In [ ]:
gapminder1.columns

In [ ]:
gapminder1.dtypes

In [ ]:
gapminder1_pivot = gapminder1.pivot_table(index=['year','country'],
                        columns = 'measure',
                      values = 'measurement').reset_index().rename_axis(None, axis = 1)
gapminder1_pivot 

In [ ]:
# read the coffee_levels table from codeup database
sql_query = """
                SELECT * 
                From melted;
                """

melted = pd.read_sql(sql_query, db_url)

In [ ]:
melted

In [ ]:
melted_melt = melted.pivot_table(index=['date'],
                        columns = 'variable',
                      values = 'value').reset_index().rename_axis(None, axis = 1)
melted_melt